In [1]:
import pandas as pd 
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [2]:
lambda_values = [0.1, 1, 10, 100, 200]     

train_df = pd.read_csv("train.csv")
train_df

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13
0,22.6,0.06724,0.0,3.24,0.0,0.460,6.333,17.2,5.2146,4.0,430.0,16.9,375.21,7.34
1,50.0,9.23230,0.0,18.10,0.0,0.631,6.216,100.0,1.1691,24.0,666.0,20.2,366.15,9.53
2,23.0,0.11425,0.0,13.89,1.0,0.550,6.373,92.4,3.3633,5.0,276.0,16.4,393.74,10.50
3,8.3,24.80170,0.0,18.10,0.0,0.693,5.349,96.0,1.7028,24.0,666.0,20.2,396.90,19.77
4,21.2,0.05646,0.0,12.83,0.0,0.437,6.232,53.7,5.0141,5.0,398.0,18.7,386.40,12.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,16.6,0.67191,0.0,8.14,0.0,0.538,5.813,90.3,4.6820,4.0,307.0,21.0,376.88,14.81
146,32.9,0.01778,95.0,1.47,0.0,0.403,7.135,13.9,7.6534,3.0,402.0,17.0,384.30,4.45
147,36.2,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
148,11.0,7.36711,0.0,18.10,0.0,0.679,6.193,78.1,1.9356,24.0,666.0,20.2,96.73,21.52


In [3]:
X = train_df.iloc[:, 1:].to_numpy()
y = train_df['y'].to_numpy()

In [4]:

for lmbda in lambda_values:
    rmse_store = []
    for train_idx, test_idx in KFold(n_splits=10).split(X):
        X_train, X_test = X[train_idx,:], X[test_idx,:]
        y_train, y_test = y[train_idx], y[test_idx]
        model = Ridge(alpha=lmbda)
        model.fit(X_train,y_train)
        rmse = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))
        rmse_store.append(rmse)
    average = sum(rmse_store)/len(rmse_store)
    print('For lambda = {}, RMSE = {}'.format(lmbda, average))

For lambda = 0.1, RMSE = 5.5018094450578605
For lambda = 1, RMSE = 5.499838741278099
For lambda = 10, RMSE = 5.48363148607229
For lambda = 100, RMSE = 5.636642135414037
For lambda = 200, RMSE = 5.721233719861128


In [66]:
#NOT WORKING --> NO ACCESS TO VALUES OF EACH ALPHAS AND CV SCORES
clf = RidgeCV(alphas= [0.1, 1, 10, 100, 200], scoring='neg_mean_squared_error', store_cv_values=True).fit(X_train, y_train)
clf.score(X_train, y_train)

0.6886750482567847

In [5]:
ridge = make_pipeline(Ridge())
lambdas = {
    'ridge__alpha': [0.1, 1, 10, 100, 200]
}
cross_validation_estimator = GridSearchCV(
                            ridge,
                            scoring='neg_mean_squared_error',
                            refit=False,
                            cv=10,
                            n_jobs=4,
                            param_grid=lambdas,
                            return_train_score=True)
cross_validation_estimator.fit(X, y)

GridSearchCV(cv=10, estimator=Pipeline(steps=[('ridge', Ridge())]), n_jobs=4,
             param_grid={'ridge__alpha': [0.1, 1, 10, 100, 200]}, refit=False,
             return_train_score=True, scoring='neg_mean_squared_error')

In [6]:
rmse = np.sqrt(-cross_validation_estimator.cv_results_['mean_test_score'])
rmse

array([5.64676368, 5.65823897, 5.67051272, 5.88739617, 5.99140148])